In [10]:
import pandas as pd

faktor_cest = pd.read_csv('./baze/manjse/FaktorCest.csv')
nesreca = pd.read_csv('./baze/manjse/Nesreca.csv')


df = pd.merge(faktor_cest, nesreca[['ZaporednaStevilkaOsebeVPN', 'TipNesrece']], on='ZaporednaStevilkaOsebeVPN')

df = df[['StanjeVozisca', 'TipNesrece']]

# odstrani vozisca in tipe nesrec v neznanem stanju
df = df.loc[df['StanjeVozisca'] != 'OSTALO']
df = df.loc[df['TipNesrece'] != 'OSTALO']
# print("Unique values in StanjeVozisca:", df['StanjeVozisca'].unique())
# print("\nUnique values in TipNesrece:", df['TipNesrece'].unique())

# obdrži vrstice s StanjeVozisca = SUHO, ostale pa preimenuj v DRUGACNO
df.loc[df['StanjeVozisca'] != 'SUHO', 'StanjeVozisca'] = 'DRUGACNO'

df['TipNesrece_new'] = 'SE PREMIKA'
mask = (df['TipNesrece'] == 'TRČENJE V STOJEČE / PARKIRANO VOZILO') | (df['TipNesrece'] == 'TRČENJE V OBJEKT')
df.loc[mask, 'TipNesrece_new'] = 'STACIONAREN'

mask = (df['TipNesrece'] == 'BOČNO TRČENJE') | (df['TipNesrece'] == 'OPLAŽENJE') | (df['TipNesrece'] == 'NALETNO TRČENJE') | (df['TipNesrece'] == 'POVOŽENJE ŽIVALI') | (df['TipNesrece'] == 'POVOŽENJE PEŠCA') | (df['TipNesrece'] == 'ČELNO TRČENJE')
df.loc[mask, 'TipNesrece_new'] = 'SE PREMIKA'

df.drop(columns=['TipNesrece'], inplace=True)
df.rename(columns={'TipNesrece_new': 'TipNesrece'}, inplace=True)

# print("Unique values in StanjeVozisca:", df['StanjeVozisca'].unique())
# print("Unique values in TipNesrece:", df['TipNesrece'].unique())

suhoSVC = df['StanjeVozisca'].value_counts()['SUHO']
drugacnoSVC = df['StanjeVozisca'].value_counts()['DRUGACNO']
sepremikaTNC = df['TipNesrece'].value_counts()['SE PREMIKA']
stacionarenTNC = df['TipNesrece'].value_counts()['STACIONAREN']

print("STOJI:", stacionarenTNC)
# print("PREMIKA:", sepremikaTNC)
print("SUHA:", suhoSVC)
print("MOKRA:", drugacnoSVC)


ms = len(df[(df['StanjeVozisca'] == 'DRUGACNO') & (df['TipNesrece'] == 'STACIONAREN')])
ss = len(df[(df['StanjeVozisca'] == 'SUHO') & (df['TipNesrece'] == 'STACIONAREN')])
# mp = len(df[(df['StanjeVozisca'] == 'DRUGACNO') & (df['TipNesrece'] == 'SE PREMIKA')])
# sp = len(df[(df['StanjeVozisca'] == 'SUHO') & (df['TipNesrece'] == 'SE PREMIKA')])

print("\nSTOJI & SUHA:", ss, "\nSTOJI & MOKRA:", ms)

r1 = suhoSVC / ss
r2 = drugacnoSVC / ms

# print("\nRazmerje stoji & suha:", r1)
# print("Razmerje stoji & mokra:", r2)

percentage_difference = abs((r1 - r2) / ((r1 + r2) / 2)) * 100

print(f"\nKarambolov v stacionarne objekte (npr. parkiran avto, stavba, itd.) je bilo na mokri (ali poledeneli, poškodovani) cesti {round(percentage_difference, 4)}% več kot karambolov v stacionarne objekte na suhi (nepoškodovani) cesti.")

STOJI: 7158
SUHA: 22534
MOKRA: 6749

STOJI & SUHA: 5444 
STOJI & MOKRA: 1714

Karambolov v stacionarne objekte (npr. parkiran avto, stavba, itd.) je bilo na mokri (ali poledeneli, poškodovani) cesti 4.9936% več kot karambolov v stacionarne objekte na suhi (nepoškodovani) cesti.
